In [39]:
from newspaper import Article, ArticleException
from GoogleNews import GoogleNews
from concurrent.futures import ThreadPoolExecutor, as_completed
from nltk.corpus import stopwords
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import numpy as np 
import pandas as pd 
import re, string
import nltk 
import matplotlib.pyplot as plt
import seaborn as sns
from flair.models import TextClassifier
from flair.data import Sentence

from pprint import pprint
#%matplotlib inline
fsize = (16, 9)
dpi = 120
plt.rcParams.update({"figure.figsize": fsize, "figure.dpi": dpi}) # set params


INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
googlenews = GoogleNews() # create news object
search_term = "APPLE" # our search term
googlenews.get_news(search_term) # get the news
results = googlenews.results() # get the results

def get_text(inst): # download the article text for each link and save as a string
    try:
        article = Article('http://'+inst['link'])
        article.download()
        article.parse()    
        inst['text'] = article.text.strip().replace("\n"," ")
        
    except ArticleException:
        print(f"Failed on {url}")
    return inst['text']

processes = [] # multi thread the execution
with ThreadPoolExecutor() as executor:
    for i in results:
        processes.append(executor.submit(get_text, i))

In [30]:
def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

def lemmatize_sentence(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = []
    for word, tag in pos_tag(tokens):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatized_sentence.append(lemmatizer.lemmatize(word, pos))
    return lemmatized_sentence
